In [45]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

from env_class import BatteryManagementEnv

plt.style.use("ggplot")

In [46]:
env = BatteryManagementEnv()
env.reset()

array([50.], dtype=float32)

In [47]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space 1
State Space 1


In [48]:
print(env.nA)

2


In [49]:
from keras.layers import Dense, Input
from keras.models import Model

#build neural network for DQN
def build_model(states, actions):
    input = Input(shape=(1,states))
    x = Flatten()(input)
    x = Dense(16, activation='relu')(x)
    #output layer
    output = Dense(actions, activation='linear')(x)

    model = Model(inputs=input, outputs=output)

    return model

model = build_model(env.observation_space, env.nA)

In [50]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()
from tensorflow.keras.optimizers.legacy import Adam

# Then, define DQN agent in Keras-RL
memory = SequentialMemory(limit=20000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy,
                nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [51]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, nb_max_episode_steps=200, log_interval=1000)

Training for 100000 steps ...
Interval 1 (0 steps performed)
   1/1000 [..............................] - ETA: 2:16 - reward: 100.000

c:\Users\janle\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


1000/1000 [==============================] - 10s 10ms/step - reward: 8.6000
1000 episodes - episode_reward: 8.600 [-100.000, 100.000] - loss: 1041.050 - mae: 50.216 - mean_q: 70.615 - mean_eps: 0.932

Interval 2 (1000 steps performed)
1000/1000 [==============================] - 18s 18ms/step - reward: 13.2000
1000 episodes - episode_reward: 13.200 [-100.000, 100.000] - loss: 0.000 - mae: 50.000 - mean_q: 100.000 - mean_eps: 0.865

Interval 3 (2000 steps performed)
1000/1000 [==============================] - 19s 19ms/step - reward: 19.6000
1000 episodes - episode_reward: 19.600 [-100.000, 100.000] - loss: 0.000 - mae: 50.000 - mean_q: 100.000 - mean_eps: 0.775

Interval 4 (3000 steps performed)
 339/1000 [=========>....................] - ETA: 11s - reward: 25.0737done, took 52.931 seconds
